In [1]:
#import thinkplot
#import thinkstats2
import pandas as pd
import numpy as np
import math
import scipy.stats as ss
import thinkplot
import thinkstats2

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

ModuleNotFoundError: No module named 'scipy'

### Dear Akeem, I originally did this in a different github environment since this once gives me a lot of trouble with the libraries. 

<h1>Assignment 2: Life Expectancy and You!</h1>

<b>Instructions:</b>
<ul>
<li>Use the worksheet below to act as a guide to exploring and examining the data in the dataset.
<li>Try to make things easy to read. Look at the formatting stuff above. 
<li>As long as you do what is asked, you can do things in many different ways. You may need to do a little searching. 
</ul>

### Please Use Formatting to Make it Clear What I Should be Looking at! Please!!! I Beg You!!!

In [2]:
#Load data and do some cleanup before starting
#Please don't change this stuff. 
dfLE = pd.read_csv("led.csv")
dfLE.rename(columns={"Status":"isDeveloped", }, inplace=True)
dfLE["isDeveloped"].replace(('Developed', 'Developing'), (1, 0), inplace=True)
dfLE = dfLE.drop(columns={"Year", "Incomecompositionofresources", "thinness5-9years", "HIV/AIDS", "percentageexpenditure", "Totalexpenditure", "thinness1-19years"})
dfLE = dfLE.groupby("Country", as_index=False).mean()
dfLE.head()


,Country,isDeveloped,Lifeexpectancy,AdultMortality,infantdeaths,Alcohol,HepatitisB,Measles,BMI,under-fivedeaths,Polio,Diphtheria,GDP,Population,Schooling
0,Afghanistan,0.0,58.19375,269.0625,78.2500,0.014375,64.562500,2362.2500,15.51875,107.5625,48.3750,52.3125,4.136734e+09,9.972260e+06,8.21250
1,Albania,0.0,75.15625,45.0625,0.6875,4.848750,98.000000,53.3750,49.06875,0.9375,98.1250,98.0625,1.166938e+09,6.969116e+05,12.13750
2,Algeria,0.0,73.61875,108.1875,20.3125,0.406667,78.000000,1943.8750,48.74375,23.5000,91.7500,91.8750,7.345597e+10,2.164983e+07,12.71250
3,Angola,0.0,49.01875,328.5625,83.7500,5.740667,70.222222,3561.3125,18.01875,132.6250,46.1250,47.6875,2.177835e+10,1.014710e+07,8.04375
4,AntiguaandBarbuda,0.0,75.05625,127.5000,0.0000,7.949333,98.266667,0.0000,38.42500,0.0000,96.9375,98.3125,0.000000e+00,NaN,8.84375


<h2>Part 1: Add a column for GDP per Capita (5pts)</h2>

Show print the labels along with just that column. Print 10 random rows of that data. 

Note: you'll have to look at the units to ensure you get it right. 

In [ ]:
#Part 1
# The 47 countries without data on Population
dfLE[dfLE['Population'].isnull()]['Country']

In [ ]:
# No country without GDP data
dfLE['GDP'].isnull().sum()

In [ ]:
# Creating dfLE without the rows with null values of population
dfLE_ready = dfLE[~dfLE['Population'].isnull()]
dfLE_ready.head(10)

In [ ]:
# Printing the first ten of the rows where the GDP-per-capita column is created
dfLE['GDP_per_capita'] = ''
dfLE['GDP_per_capita'] = dfLE_ready['GDP']/dfLE_ready['Population']
dfLE.head(10)

# The cell just below delivers what is asked in Part 1: ten (10) random rows of the column created for GDP per capita

In [ ]:
dfLE.sample(10)[['Country', 'GDP_per_capita']]

### Part 2: Print a visualization of each distribution. Both regular (density) and cumulative. (15pts)

Print each distribution is some type of visualization, also print the cumulative distribution. 
<ul>
<li>5 points are for using some visualization that is effective.
<li>5 points for formatting things in a nice layout. Hint: for whatever chart you use, look into some formatting options. Think of what to try to Google for an example for printing multiple charts, a grid of charts, etc... and try to adapt one to yours. There is also a way to do it using some thinkstats/thinkplot stuff. 
<li><b>5 points are for doing it in an efficient way - just listing off each column is lots of typing... Remember, the #1 trait of programmers is laziness with respect to repetitive tasks. If we are doing the same thing over and over with tiny changes, try to think of a condensed way to do this...</b>
</ul>

In [ ]:
#Print all distributions
#Filled all null values with the median
dfLE_filled = dfLE.fillna(dfLE.median(numeric_only=True))
dfLE_filled.head(10)

In [ ]:
dfLE_filled_CountryNameRemoved = dfLE_filled.select_dtypes(include=['number'])
dfLE_filled_CountryNameRemoved

In [ ]:
#Print all distributions
columns_to_plot = dfLE_filled_CountryNameRemoved.columns.tolist()

numRows = int(np.ceil(len(columns_to_plot) / 4)) 
fig, axes = plt.subplots(numRows, 4, figsize=(20, 5*numRows))
axes = axes.flatten()
for i, col in enumerate(columns_to_plot):
    sns.histplot(data=dfLE_filled_CountryNameRemoved, x=col, ax=axes[i], bins = 50, kde = True) 
    axes[i].set_title(f'histogram for : {col}')
    axes[i].set_xlabel('value')

In [ ]:
columns_to_plot = dfLE_filled_CountryNameRemoved.columns.tolist()

numRows = int(np.ceil(len(columns_to_plot) / 4)) 
fig, axes = plt.subplots(numRows, 4, figsize=(20, 5*numRows))
axes = axes.flatten()
for i, col in enumerate(columns_to_plot):
    sns.ecdfplot(data=dfLE_filled_CountryNameRemoved, ax=axes[i], x=col)
    axes[i].set_title(f'CDF for : {col}')
    axes[i].set_xlabel('value')

### Create an analytical distribution for SCHOOLING, and use it for a couple of things. (20pts)

<ul>
<li>5 points are for choosing an appropriate distribution. Please state it clearly somewhere.
<li>10 points are for creating and visualizing the distribution. Do it <b>without</b> using a fit() method. Plot the emperical data on the same graph as the analytical distribution. 
<li>5 points are for using the distribution to predict the percentage of that have a schooling level within 1 year of Canada. 
</ul>

In [ ]:
#Empirical for Schooling

sns.histplot(data=dfLE['Schooling'], bins = 10, kde=True)

print('median', dfLE['Schooling'].median())
print('mean', dfLE['Schooling'].mean())

# Schooling histplot is almost normal but median > mean. It's really skews to the left by a bit.

In [ ]:
# Schooling to the power of pwr
pwr = 1
dSchPwrd = dfLE['Schooling']**pwr
while dSchPwrd.median() > dSchPwrd.mean():
    dSchPwrd = dfLE['Schooling']**pwr
    pwr=pwr+0.001

print('school years raised to the power of', round(pwr, 2))    
print('median', dSchPwrd.median())
print('mean', dSchPwrd.mean())

sns.histplot(dSchPwrd, bins=20, kde=True) # more normal empirical distro: median and mean almost equal. 

In [ ]:
# Canada's schooling is 15.875 years. We'll then compute percentage of countries with schooling between 15.375 and 16.375. These are within 1 year of Canada's
print(dfLE_filled[dfLE_filled['Country']=='Canada']['Schooling'])

In [ ]:
#Empirical for Canada-like Schooling

LessThanLow = dfLE[dfLE['Schooling'] < 15.375]['Schooling'].count() # number of countires less than lower limit for Schooling
LessThanEqualHigh = dfLE[dfLE['Schooling'] <= 16.375]['Schooling'].count() # number of countires less than or equal to the upper limit for Schooling
NotNullSchooling = dfLE[dfLE['Schooling'].isnull() == False]['Country'].count() # Countries with values for Schooling, not null.
CanSchoolEmpirical = 100*(LessThanEqualHigh - LessThanLow)/NotNullSchooling

print(LessThanEqualHigh)
print(LessThanLow)
print(NotNullSchooling)
print(CanSchoolEmpirical)

# Empirically, there's a 7.30% chance that a random country in this dataset will have a comparable number of years of schooling with Canada's (15.875 years), i.e. give or take half a year.

### Analytical for Schooling: generating random values given mean and std of empirical data

In [ ]:
analyticaldata = ss.norm.rvs(size=100000, scale=np.std(dfLE['Schooling']**pwr), loc=np.mean(dfLE['Schooling']**pwr))
print(analyticaldata)
sns.histplot(data=analyticaldata, kde=True)

In [ ]:
### Analytical for Schooling: counting how many of the randomly generated values were within one year of Canada's schooling years
count=0
i = 0
while i < len(analyticaldata):
    if analyticaldata[i] <= (16.375**pwr) and analyticaldata[i] >= (15.375**pwr):
        count = count + 1
        i=i+1
    else:
        i=i+1

CanSchoolAnalytical = 100*count/len(analyticaldata)
print('Analytical percentage', CanSchoolAnalytical)
print('% error', 100*abs(CanSchoolAnalytical-CanSchoolEmpirical)/CanSchoolAnalytical)

### Analytically, there's a 6 to 7 percent chance that a random country in our analytical distro has total years of schooling that is within half a year more or less than Canada's 15.875 years.

### Part 3: Create another analytical distribution for GDP per Capita (20pts)

<ul>
<li>5 points are for choosing an appropriate distribution. Please state it clearly somewhere.
<li>10 points are for creating and visualizing it alongside the emperical. You have free reign on how. 
<li>5 points are for using the analytical distribution to evaluate the quality of your GDP per capita metric - does it appear accurate? Why or why not?
<li>If you've judged GDP per cap to be unreliable, drop it. 
</ul>

In [ ]:
#Empirical for GDP/capita
dGDPcap = dfLE['GDP_per_capita'].dropna()
sns.histplot(data=dGDPcap, bins = 50, kde=True)

print('median', dGDPcap.median())
print('mean', dGDPcap.mean())
print('mean squared', np.square(dGDPcap.mean()))
print('variance', dGDPcap.var())
print('skewness', ss.skew(dGDPcap))

# Definitely right-skewed, meriting log transformation.  Its variance is very different from the square of its mean. Its skewness is far from 2. So, not exactly exponential. 

In [ ]:
#Empirical for GDP/capita: checking for log-normality
islognormal = np.log(dGDPcap)
sns.histplot(islognormal, bins = 50, kde =True)
print('mean', islognormal.mean())
print('median', islognormal.median())
print('skew', ss.skew(islognormal))


In [ ]:
#Analytical for GDP/capita
# Here we generate analytical data for a lognormal distribution.
# we superimpose the analytical and empirical distribution together
yeslognormal_GDPcapita = np.random.lognormal(size=10000, mean=islognormal.mean(), sigma=islognormal.std())
print(yeslognormal_GDPcapita)
sns.ecdfplot(yeslognormal_GDPcapita)
sns.ecdfplot(dGDPcap)

In [ ]:
#Analytical for GDP/capita
statistic, p_value = ss.kstest(islognormal, 'norm', args=(islognormal.mean(), islognormal.std()))
print(f"K-S Statistic: {statistic}") 
print(f"P-value: {p_value}")

### The P value is insignificant, way beyond 0.05. The K-S Statistic is also close to zero. These corroborate what's shown in the cdf graph of the analytical and empirical data above: the random values generated are very much like our original data. I'd keep GDP per capita.

# GDP per capita is more lognormal than exponential. The skew of the natural log of the dataset is close to zero. The mean and median of the natural log of the dataset are almost equal. So, lognormal it is.

### Part 4: Visually identify correlations. Separate developed and not developed countries is some way. (15pts)

<ul>
<li>5 points are for effectively and clearly showing the visualizations used to evaluate correlations.
<li>10 points are for, in the process of visualizing, remove outliers that confound correlations. List outlier "filters" you used and justify why - was the data an error, a real value that is just way out of line, or somethingn else? Remember, you're trying to find things that have correlations with each other. Looking at a correlation between home size and income may not benefit from Jeff Bezos being included. It may be helpful to look at the results and revise. 
</ul>

In [ ]:
#separating developed from developing countries
dfLE_developed = dfLE_filled[dfLE_filled["isDeveloped"] == 1]
dfLE_notDeveloped = dfLE_filled[dfLE_filled["isDeveloped"] == 0]

In [ ]:
#Generating pairplot for the developed countries dataset
sns.pairplot(dfLE_developed, dropna=True, kind = 'reg')
plt.show()

### Looking at the plots under the isDeveloped columns, some dots are really far from the clustering of all other dots. The rows for Diptheria, underfiveDeaths, BMI, Measles, Hepa B, Alcohol and infantDeaths all have outliers. You can pass a straight horizontal line through these outlying dots and skewer the same dot across all columns.  An upper or lower cutoff value will be set for each of these features. Same will be done for developping countries dataset below. We might have new outliers after the initial pruning but we can't just prune the data ad nauseam.

In [ ]:
## df2 = df2[(df2["height"]>110) & (df2["height"] < 200)] -- taken from our class examples
# Removing those outliers, we get the new pairplots below.
dfLE_developed = dfLE_developed[(dfLE_developed['Diphtheria'] > 80) & (dfLE_developed['under-fivedeaths'] < 30) & (dfLE_developed['BMI'] > 25) & (dfLE_developed['BMI'] < 65) & (dfLE_developed['Measles'] < 6875.8125) & (dfLE_developed['HepatitisB'] > 60) & (dfLE_developed['Alcohol'] > 5) & (dfLE_developed['infantdeaths'] < 20)]
sns.pairplot(dfLE_developed, dropna=True, kind = 'reg')
plt.show()

In [ ]:
#Generating pairplot for developing countries dataset
sns.pairplot(dfLE_notDeveloped, dropna=True, kind = 'reg')
plt.show()

In [ ]:
## df2 = df2[(df2["height"]>110) & (df2["height"] < 200)] -- taken from our class examples
# Removing those outliers, we get the new pairplots below.
dfLE_notDeveloped = dfLE_notDeveloped[(dfLE_notDeveloped['GDP'] < 200000000000) & (dfLE_notDeveloped['GDP_per_capita'] < 15000) & (dfLE_notDeveloped['Population'] < 200000000) & (dfLE_notDeveloped['under-fivedeaths'] < 1500) & (dfLE_notDeveloped['infantdeaths'] < 5000) & (dfLE_notDeveloped['Measles'] < 4000)]
sns.pairplot(dfLE_notDeveloped, kind = 'reg')

### Part 5: Calculate correlations (25pts)

<ul>
<li>5 points are for identifying and calculating 3 strongest correlations with life expectancy for each of developed and not developed. 
<li>10 points are for analyzing those correlations. Do they appear to be supported by reality? Do you have reason to beleive any are causal?
<li>10 points are for identifying 2 places where developing and developed countries have a substantially different relationship with respect to life expectancy, and theorizing why. What would be needed to confirm your thoughts?
</ul>

<b>Please put your written parts in markdown cells, with a little formatting to make them readable. 

In [ ]:
#Calculate correlations and analyze. developed  countries
print(dfLE_developed.corr(numeric_only=True))

### For developed countries, Lifeexpectancy is most strongly correlated to GDP_per_capita, GDP, and Schooling. It is most negatively correlated to AdultMortality.

In [ ]:
#Calculate correlations and analyze. developing countries
print(dfLE_notDeveloped.corr(numeric_only=True))

### For developing countries, Schooling, Diphtheria, and Polio are the three most highly correlated with Lifeexpectancy. It is most negatively correlated to AdultMortality.

### Schooling and Lifeexpectancy go hand in hand in both developed and developing countries. But among developing countries, Schooling and these two vaccines (Diphtheria and Polio) and that for Hepa B also correlate among themselves. Unlike for developed countries, GDP and GDP per capita do not correlate with Life expectancy. A reasonable explanation that countries that spend for public services like Schooling and vaccines, including developing countries not having as much werewithal as developed countries, increase Lifexpectancy of their citizens. This is just one case perhaps that show why care must be taken when extrapolating causation from mere correlation. I'd be cautious against claims that these correlations are causal.

### Counterintutively, Diphtheria and Polio are negatively correlated to Lifeexpectancy among developed countries. This is the total opposite among developing countries. 